In [ ]:
import pandas as pd
import numpy as np
from conexao import conn, desconecta

connection = conn()

try:
    query = connection.cursor()
    query.execute("SELECT * FROM empresa")
    dados = query.fetchall()
    colunas = [desc[0] for desc in query.description] 
    df_empresa = pd.DataFrame(dados, columns=colunas)
    
    print(df_empresa.head())
    
    desconecta(connection)
except Exception as e:
    print("Ocorreu um erro inesperado.")

UndefinedTable: relation "empresa" does not exist
LINE 1: SELECT * FROM empresa
                      ^
